# Worksheet 08

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.784208388776837, 5.454749029093103, 4.83448796560567, 4.457465714923971, 4.276354112051536, 4.672357114590641, 6.40048407711412, 5.287558207740122, 6.570378517437532, 5.329371844299757]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
s2 = np.random.normal(8 , 1, 10).tolist()
print(s2)

[5.624278880540896, 9.138435410943696, 9.150321001324368, 7.392630916074368, 7.649772025129963, 8.722007067100698, 9.408098801483714, 7.303670951602387, 8.942826625366061, 7.545419193131037]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[7.545419193131037, 8.942826625366061, 7.303670951602387, 9.408098801483714, 8.722007067100698, 5.329371844299757, 7.649772025129963, 6.570378517437532, 5.287558207740122, 6.40048407711412]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

mu1, sd1, mu2, sd2

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.545419193131037, 8.942826625366061, 7.303670951602387, 9.408098801483714, 8.722007067100698, 7.649772025129963]
[5.329371844299757, 6.570378517437532, 5.287558207740122, 6.40048407711412]
P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 8.26196577730231,  mean_2 = 5.896948161647883
var_1 = 0.6325632603857145,  var_2 = 0.3501389635503895


The actual means were 8 and 5, which are relatively close to the estimates. The estimated variances were both a little low.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))

    if(p[1] > p[2]):
        print("P(S_1) > P(S_2)")
    else:
        print("P(S_2) > P(S_1)")


point =  7.545419193131037
probability of observing that point if it came from cluster 0 =  0.33202544827604774
probability of observing that point if it came from cluster 1 =  1.751643203351435e-05
point =  8.942826625366061
probability of observing that point if it came from cluster 0 =  0.35337321992479054
probability of observing that point if it came from cluster 1 =  4.210768902917292e-17
point =  7.303670951602387
probability of observing that point if it came from cluster 0 =  0.20019092417625245
probability of observing that point if it came from cluster 1 =  0.0003561638186336372
point =  9.408098801483714
probability of observing that point if it came from cluster 0 =  0.12215917378502483
probability of observing that point if it came from cluster 1 =  1.6622879653734864e-22
point =  8.722007067100698
probability of observing that point if it came from cluster 0 =  0.4841207731578344
probability of observing that point if it came from cluster 1 =  8.329279283122855e-15
point

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 0.4
mean_1 = 8.21770387999252,  mean_2 = 5.870944657558112
var_1 = 0.691046681899297,  var_2 = 0.3472779209262448


Basically no changes as of yet.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [10]:
prob_x = []
prob_s0_x = []
prob_s1_x = []

k = 2

for p in data:
    pdf_i = []

    for j in range(k):
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        prob_s[j]

    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = list(zip(data, prob_s0_x, prob_s1_x))

for p in probs:
    print(p[0])
    print("P(S_1) = " + str(p[1]))
    print("P(S_2) = " + str(p[2]))
    print()

7.545419193131037
P(S_1) = 0.9999809537971592
P(S_2) = 1.9046202840800353e-05

8.942826625366061
P(S_1) = 1.0
P(S_2) = 2.3508110989347643e-17

7.303670951602387
P(S_1) = 0.9993596719478407
P(S_2) = 0.0006403280521593158

9.408098801483714
P(S_1) = 1.0
P(S_2) = 1.7372046542149092e-22

8.722007067100698
P(S_1) = 0.999999999999996
P(S_2) = 4.00576654767317e-15

5.329371844299757
P(S_1) = 0.000409011456388844
P(S_2) = 0.9995909885436111

7.649772025129963
P(S_1) = 0.999996266432143
P(S_2) = 3.733567856995326e-06

6.570378517437532
P(S_1) = 0.2505454850111253
P(S_2) = 0.7494545149888746

5.287558207740122
P(S_1) = 0.000385311271745844
P(S_2) = 0.9996146887282541

6.40048407711412
P(S_1) = 0.07059215288088551
P(S_2) = 0.9294078471191144



No changes.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [11]:
for p in probs:
    if(p[1] > p[2]):
        print("Data point " + str(p[0]) + " is assigned to cluster 0")
    else:
        print("Data point " + str(p[0]) + " is assigned to cluster 1")

Data point 7.545419193131037 is assigned to cluster 0
Data point 8.942826625366061 is assigned to cluster 0
Data point 7.303670951602387 is assigned to cluster 0
Data point 9.408098801483714 is assigned to cluster 0
Data point 8.722007067100698 is assigned to cluster 0
Data point 5.329371844299757 is assigned to cluster 1
Data point 7.649772025129963 is assigned to cluster 0
Data point 6.570378517437532 is assigned to cluster 1
Data point 5.287558207740122 is assigned to cluster 1
Data point 6.40048407711412 is assigned to cluster 1
